- IMGRGB_0112.xlsx와 TFres_0112re.csv을 기반으로 수행한다  // Slice_img05.ipynb
더하여 처리 안된 이미지는 수동으로 작업함

seperIMG_notM02.ipynb

- 01/13
이미지 처리부터 싹다 다시
- 01/16


/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori 


이미지 재 split

#import library

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

In [ ]:
!pip install callee
#for excuse error

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

from PIL import Image
import os
from glob import glob
import shutil
from callee import *
import math

import shutil#이미 실행 시킴_초기화 구현 전에는 수행하지 말 것
import random

#dir list
- (01/16) 총 합 이미지의 경우, "/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01" 하위에 unmasked와 masked로 구분되어 있음

In [ ]:
#about oriimg
base_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/" #unmask+mask img slies
ORIImg="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_orig/tumor_prev/" #WSI이미지
prev_slice=base_dir+"ALL/"#WSI 이미지 slice 저장
oribase_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/DeepL/"

#about 2img
ori_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/"
#about learningimg
test_dir=base_dir+"test/"
train_dir=base_dir+"train/"
validation_dir=base_dir+"validation/"

#IMG slice


##slice img 

In [ ]:
w_num=30 ; h_len=30#image slice # (갯수로 지정한다)

def image_crop( infilename ,save_path): #( infilename ,prev_path ,save_path)
    img = Image.open(ORIImg+infilename )
    (img_h, img_w) = img.size #width, height  ;#print(img.size)

    wslice_size=(img.width)/w_num
    hslice_size=(img.height)/h_len

    grid_w = wslice_size # crop width
    grid_h = hslice_size # crop height
    range_w = (int)(img_w/grid_w)
    range_h = (int)(img_h/grid_h)
 
    i = 0
    for w in range(range_w):
        for h in range(range_h):
            bbox = (h*grid_h, w*grid_w, (h+1)*(grid_h), (w+1)*(grid_w))   # 가로 세로 시작, 가로 세로 끝
            crop_img = img.crop(bbox)
 
            fname = "{}".format("{0:04d}".format(i))
            savename = save_path + fname +"_"+infilename
            
           ## crop_img.save(savename)
            print('save file ' + savename)
            i += 1

In [ ]:
for (path,dir,file) in os.walk(ORIImg):
  #if(path.rfind("masked")!=-1):
  print(path,dir)
  if(len(dir)==0):
    infilename=path[int(path.rfind("/"))+1:] ; 
    print("resr",infilename)
    image_crop(infilename, prev_slice)

##delete img 1KB 미만 + unmaksed/ori에서 분리
- 확인 결과 전혀 상관 없는 이미지의 경우 2kb미만안 경우가 대다수
- (01/16) masked를 통해 분리 후 unmasked로 분리


masked !find ./* -size -1899c -size +1c 

In [ ]:
## find . -size 0 -exec rm -rf {} \;0002_87-1+original
#%cp /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/Masked/* /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/MNormal/
#%mkdir IMG_ALL
%cd  /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/UnMCancer/
#%ls -lSh
#%pwd
#%rm -r *
#!find ./* -size -1899c -size +1c  -exec rm -rf {} \;

##!find ./* -size -1800c -size +1c 

# for (path,dir,file) in os.walk(Img):
#   if (len(dir)==False):# and (path.rfind("masked")!=-1):
#     print(path,dir)
#     infilename=path[int(path.rfind("/"))+1:] ; print("infilename",infilename)
#     #image_crop(infilename,base_dir+"result/") 

##data sep

In [ ]:
re_ftotal=[""]#total에서 normal img list

for (path,dir,file) in os.walk(mori_dir+"mtotal"):
  #print(cpath,cdir,cfile)#sk_slice_img/mtotal [] ['0421_74-2+masked.jpg', '0
  #if (len(dir)!=False):# and (path.rfind("masked")!=-1):
  for (cpath,cdir,cfile) in os.walk(mori_dir+"mtotal_cancer"):
    for (cpath,cdir,nfile) in os.walk(mori_dir+"mtotal_normal"):
      for i in file:
        if i not in nfile:
          re_ftotal.append(i)
      del re_ftotal[0]

    print('cfile',cfile,'/ nfile',nfile)
    print("re_total",re_ftotal)
    print("file",file)
  print(len(file),len(cfile),'/',len(re_ftotal),'/',len(nfile))
    #infilename=path[int(path.rfind("/"))+1:] ; print("infilename",infilename)
    #image_crop(infilename,base_dir+"result/") 

In [ ]:
#작업
#img_mv(mori_dir+"mtotal",mori_dir+"mtotal_normal",re_ftotal)

#Data Setup
- ( cancer mask )는 가능한 포괄적으로 선택하는 것으로
- result에서 normal cancer 복붙 시행
normal의 경우 masked된 자료는 어느정도 정리 완료

In [ ]:
# for name in ctf["name_value"]:
#   shutil.copy(slice_ori+name, slice_fil)   #(source, destination)

#print(len(os.listdir(base_dir+"result_ori/")),'==>',len(os.listdir(Img))) #10410 ==> 6427

#%cd /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/IMGset/result/
#!find /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/IMGset/result/* -size -4000c -size +1c -exec cp /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/IMGset/result/ /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/IMGset/result_normal/ {} \;

#print(len(os.listdir("/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/MCancer/")))


# for (path,dir,file) in os.walk(Img+"result_normal"):
#   if (len(dir)==0):# and (path.rfind("masked")!=-1):
    
#     infilename=path[int(path.rfind("/"))+1:]
#     print("infilename",infilename)
    #image_crop(infilename,base_dir+"result/") 

- 6425 infilename result_cancer
- 6425 infilename result_normal


##masked img에서 분류해둔 info를 통해 unmaksed img와 매칭 시킨다.(0116) 
- Img+'result'에 있는 orignal img(unmaksed img) 에서 추출

In [ ]:
def img_mv2(b_dir,dir_name,st_list,list_name):
    #clear #insert
    print("list_name:",list_name,"\nst_list:",st_list)
    for fn in list_name:
      try:
          #if len((os.listdir(dir_name)))==0 :
          #print('b_dir :',b_dir+fn, "dir_name :",dir_name,'\n')
      #    print('b_dir :',b_dir+fn)
          print('dir_name :',dir_name+fn)
          #shutil.copy(b_dir+fn,dir_name+fn)#copy file
      except:
        continue

In [ ]:
tm_list=os.listdir(ori_dir+"MCancer/")
nm_list=os.listdir(ori_dir+"MNormal/")
tum_list=[""];num_list=[""];tt_list=[""]

# for i in tm_list:
#   tum_list.append(i.split("+")[0]+"+original.jpg")
#   tt_list.append(i.split("+")[0])
# del tum_list[0]
# del tt_list[0]
# print(tt_list)
# img_mv2(ori_dir+"UnMasked/",ori_dir+"UnMCancer/",tt_list,tum_list)

tt_list=[""]
for i in nm_list:
  num_list.append(i.split("+")[0]+"+original.jpg")
  tt_list.append(i.split("+")[0])
del num_list[0]
del tt_list[0]
img_mv2(ori_dir+"UnMasked/",ori_dir+"UnMNormal/",tt_list,num_list)

#modeling

##data

In [ ]:
def rm_list(tlist,tmdir):
  print("함수")
  print(tlist,"\n",tmdir)
#  for x in tlist:
#    print(x+"+masked.jpg")
#  for x in tlist:
#    #os.remove(tmdir+x)
  return 0

In [ ]:
tum_list=[""];num_list=[""];tt_list=[""]
for i in uncm_list:
  tum_list.append(i.split("+")[0])
del tum_list[0]

for i in unnm_list:
  num_list.append(i.split("+")[0])
del num_list[0]
#print(len(tum_list),"tum_list :",tum_list) ;print(len(num_list),"num_list :",num_list);#1069 tum_list : ['0246_141-1', '0246_87-1' // 1492 num_list : ['0602_127-1', '0602_74-2', 

In [ ]:
#%ls /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/
# %cd /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/
# %ls -ls
# %cd /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/train/

#about learningimg
test_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/test/"
train_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/train/"
validation_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/validation/"

In [ ]:
def img_mv(b_dir,dir_name,list_name):#정리 하기
  #clear 
  if len((os.listdir(dir_name)))==0 :
    print('b_dir :',b_dir,"dir_name :",dir_name)
    for fn in list_name:
#      print("fn",fn)     
      #shutil.copy(b_dir+fn,dir_name+fn)#copy file

In [ ]:
def rm_list(Ttlist,aplist):
  tmplist=[""]
  for x in Ttlist:
    try:
      if x not in aplist:
        tmplist.append(x)
    except:
      continue
  del tmplist[0]
  print(len(tmplist),"//tmplist :",tmplist)
  return tmplist

In [ ]:
# tr:vl:te=6:2:2  #image size (128, 128)
uncm_list=os.listdir(base_dir+"UnMCancer/")
unnm_list=os.listdir(base_dir+"UnMNormal/")
# print(len(tm_list),"tm_list",tm_list);print(len(nm_list),"nm_list",nm_list)
tmpnm=[""];tmpcm=[""];

#test_dir,train_dir,validation_dir
cm_list=[""];nm_list=[""];

#cm_list=random.sample(uncm_list,int(len(uncm_list)*0.6))
#img_mv(base_dir+"UnMCancer/",train_dir+"cancer/",cm_list)
#tmpcm=rm_list(uncm_list,cm_list) ;
#nm_list=random.sample(unnm_list,int(len(unnm_list)*0.6))
#img_mv(base_dir+"UnMNormal/",train_dir+"normal/",nm_list)
#tmpnm=rm_list(uncm_list,nm_list) ;

#cm_list=random.sample(uncm_list,int(len(uncm_list)*0.2))
#img_mv(base_dir+"UnMCancer/",test_dir+"cancer/",cm_list)
#tmpcm.append(rm_list(uncm_list,cm_list)) 
#nm_list=random.sample(unnm_list,int(len(unnm_list)*0.2))
#img_mv(base_dir+"UnMNormal/",test_dir+"normal/",nm_list)
#tmpnm.append(rm_list(tmpnm,nm_list)) ;

#cm_list=random.sample(uncm_list,int(len(uncm_list)*0.2))
#img_mv(base_dir+"UnMCancer/",validation_dir+"cancer/",cm_list)
#nm_list=random.sample(unnm_list,int(len(unnm_list)*0.2))
#img_mv(base_dir+"UnMNormal/",validation_dir+"normal/",nm_list)
#print(len(uncm_list),'//',len(tmpcm),"tmpcm",tmpcm)
#print(len(unnm_list),'//',len(tmpnm),"tmpnm",tmpnm)

##model

In [ ]:
from tensorflow.keras.layers import Dense

img_size=128#이는 확인된것
    #CNN modeling
model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='sigmoid', input_shape=(img_size, img_size, 3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.4,seed=None),#addnoise_shape=True, 
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # #tf.keras.layers.Dense(512, activation='relu'),#,kernel_initializer='random_normal'
      #tf.keras.layers.Dense(1, activation='softmax'),
      tf.keras.layers.Dropout(0.2),#, noise_shape=None
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      ]) 

      # tf.keras.layers.MaxPooling2D(2,2),
      # tf.keras.layers.Dropout(0.6, noise_shape=None, seed=None),
      # tf.keras.layers.Flatten(),
      # #tf.keras.layers.Dense(512, activation='relu'),#,kernel_initializer='random_normal'
      # tf.keras.layers.Dense(1, activation='softmax') 
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=3, activation='sigmoid'))#sigmoid
model.add(tf.keras.layers.Flatten())
# model compile: 
model.compile(loss='binary_crossentropy', optimizer='Adamax')

print(model.summary())

##model 2
- (01/17 )기본 설정 값 test_generator ,batch_size =20 / try_t=0 ;steps_per_epoch_n=30; epochs_n=200;batch_size_n=50; validation_steps=50
tmp_ep=0.0
- train_datagen.flow_from_directory batch_size=300, / try_t=0 ; epochs_n=300;batch_size_n=100; validation_steps=10#;steps_per_epoch_n=50
tmp_ep=0.0

In [115]:
from tensorflow.keras.layers import Dense

img_size=300

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()
# model compile: 
# model.compile(loss='binary_crossentropy', optimizer='Adam')

print(model.summary())

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_95 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_90 (MaxPoolin  (None, 149, 149, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_96 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_91 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_97 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_92 (MaxPoolin  (None, 35, 35, 64)     

In [116]:
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#model complie
model.compile(optimizer=RMSprop(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

#img pre
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( rescale = 1.0/200. )
test_datagen  = ImageDataGenerator( rescale = 1.0/200. )

#/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/
# train_generator = train_datagen.flow_from_directory(train_dir,
#             batch_size=100,  class_mode='binary',target_size=(img_size, img_size))
# validation_generator = test_datagen.flow_from_directory(validation_dir,
#             batch_size=100, class_mode='binary',target_size=(img_size, img_size))
# #test_datagen
# test_generator = test_datagen.flow_from_directory(test_dir,
#             batch_size=100, class_mode='binary',target_size=(img_size, img_size))

m_ori_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/"
#m_ori_dir+"UnMasked", Image prep
train_generator = train_datagen.flow_from_directory(train_dir,
            batch_size=300,  class_mode='binary',target_size=(img_size, img_size))
#test_datagen
test_generator = test_datagen.flow_from_directory(test_dir,
            batch_size=100, class_mode='binary',target_size=(img_size, img_size))

validation_generator = test_datagen.flow_from_directory(validation_dir,
            batch_size=100, class_mode='binary',target_size=(img_size, img_size))

Found 1536 images belonging to 2 classes.
Found 511 images belonging to 2 classes.
Found 511 images belonging to 2 classes.


In [ ]:
import pandas as pd
import os
#결과 저장하기 -- 높은 acc를 위해서
#print("train_dir : ",train_dir); #print("os.listdir : ",nm_list) #train_dir :  /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/DeepL/train/  # os.listdir :  ['31-1_0.4x_47.jpg',
#print(history.history.keys())

Train_list,Validation_list=list(),list()
tmp_num01=0
#/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/DeepL/validation/tumor_validation/ 

def save_res(try_t,epochs,batch_size,steps_per_epoch,Accuracy):
#model check
  Train_list,Validation_list=list(),list()
  global tmp_num01
  loss_metrics=model.evaluate(test_generator,verbose=2)#,train_generator,
  index_n=["TRY_t","Epochs","Batch_size","Steps_per_epoch","Loss","Accuracy","File_list"]#index_n=list()
  print("index_n",len(index_n))
  
  value_d=list()
  fres=open(ori_dir+"Learnigrc.csv","a")
  #dfv=pd.DataFrame();dft=pd.DataFrame()
  for (path, dir, file) in os.walk(base_dir):
    print(path);print("dir::",dir)
    if(path[-11:]=="validation"):
          print("validation ::")
          value_d=[str(try_t)+" validation",epochs,batch_size,steps_per_epoch,loss_metrics[0],loss_metrics[1]]
          dfv=pd.DataFrame([value_d],columns=[index_n])
          print("validation //",dfv)
          if(tmp_num01==0):
            dfv.to_csv(fres,header=True,mode ="a",index=None)
            tmp_num01=1
          else: 
            dfv.to_csv(fres,header=False,mode ="a")
    elif(path[-5:]=="train"):
          print("train ::")
          value_d=[str(try_t)+" train",epochs,batch_size,steps_per_epoch,loss_metrics[0],loss_metrics[1],tm_list]
          dft=pd.DataFrame([value_d],columns=[index_n])
          print("train //",dft)
          if(tmp_num01==0):
            dft.to_csv(fres,header=False,mode ="a",index=None)
            tmp_num01=1
          else: 
            dft.to_csv(fres,header=False,mode ="a")
  # pd.merge(dft,dfv,how='outer')
  # dfv.to_csv(fres,header=False,mode ="a")
    #fres.close()

In [ ]:
#training
import matplotlib.pyplot as plt
from callee import *
import traceback
import pandas as pd
import os
from tensorflow.python.keras.models import load_model

try_t=0 ; epochs_n=300;batch_size_n=100; validation_steps=10#;steps_per_epoch_n=50
tmp_ep=0.0

res_dir=ori_dir
#
while(True):
  try:
    #steps_per_epoch=10; epochs=100;batch_size=10;

    history=model.fit(train_generator,validation_data=validation_generator, epochs=epochs_n,#, steps_per_epoch=steps_per_epoch_n,batch_size=batch_size_n,
    verbose=2,shuffle=True )#validation_steps=20,
    
    batch_size=batch_size_n;steps_per_epoch=steps_per_epoch_n
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))
    print(epochs)
    tmp_ep=(sum(acc)/len(acc))
    
    if (try_t==0 or (sum(acc)/len(acc))>=tmp_ep):
      steps_per_epoch_n=steps_per_epoch_n+10;
      tmp_ep=(sum(acc)/len(acc))
      save_res(try_t,epochs,batch_size,steps_per_epoch,tmp_ep)
    elif((sum(acc)/len(acc))>=tmp_ep):
      batch_size_n=batch_size_n+10;
      #모델 저장하기#결과 값 지니고 있음
      model.save(res_dir+'mnist_mlp_model.h5')
      save_res(try_t,epochs,batch_size,steps_per_epoch,tmp_ep)
      #model res import 
      tmpload_md=tf.keras.models.load_model(res_dir+'mnist_mlp_model.h5')
      #test_loss, test_acc = tmpload_md.evaluate(train_generator,validation_data=validation_generator, verbose=2)#train_generator,validation_data=validation_generator, //x,  y,
      try_t=try_t+1;
    else:
      steps_per_epoch_n=steps_per_epoch_n+10;
      batch_size_n=batch_size_n+10;
      try_t=try_t+1;

      
    
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs,val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'go', label='Training Loss')
    plt.plot(epochs,val_loss, 'r', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()
  except Exception as em:
    print(em,"// error")
    continue                    #spe<ep&spe==vs